<div class="markdown-google-sans">
  <h1>GPU Adjacency Matrix</h1>
</div>


Mount Google drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check if NVIDIA Cuda is installed.

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


**Step 1: Check GPU Compatibility**
Before installing RAPIDS, it's recommended to check if your Colab instance's GPU is compatible with RAPIDS. You can do this by executing the env-check.py script.

In [3]:
!wget https://raw.githubusercontent.com/rapidsai-community/rapidsai-csp-utils/main/colab/update_gcc.sh
!bash update_gcc.sh

--2024-03-31 13:03:31--  https://raw.githubusercontent.com/rapidsai-community/rapidsai-csp-utils/main/colab/update_gcc.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 181 [text/plain]
Saving to: ‘update_gcc.sh’

update_gcc.sh       100%[===================>]     181  --.-KB/s    in 0s      

2024-03-31 13:03:32 (13.9 MB/s) - ‘update_gcc.sh’ saved [181/181]

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Found existing installation: cupy-cuda12x 12.2.0
Uninstalling cupy-cuda12x-12.2.0:
  Successfully uninstalled cupy-cuda12x-12.2.0
restarting Colab...


In [4]:
!nvidia-smi

Sun Mar 31 13:03:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 460 (delta 131), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (460/460), 126.19 KiB | 2.63 MiB/s, done.
Resolving deltas: 100% (233/233), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 880.6 kB/s eta 0:00:00
***********************************************************************
Woo! Your instance has a Tesla T4 GPU!
We will install the latest stable RAPIDS via pip 24.2.*!  Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.7/464.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 67

In [10]:
pip install osmnx && pip install networkx && pip install pandas

In [11]:
import osmnx as ox
import networkx as nx
import pandas as pd
import cudf
import cugraph as cg
import os

In [39]:
# Function definitions
def fetch_road_network(place, network_type='bike'):
    ox.config(use_cache=True)
    graph = ox.graph_from_place(place, network_type=network_type)
    graph = ox.utils_graph.get_undirected(graph)
    return graph

def convert_nx_to_cugraph(nx_graph):
    edges_df = nx.to_pandas_edgelist(nx_graph)
    edges_df = edges_df.rename(columns={"source": "src", "target": "dst", "weight": "length"})
    edges_gdf = cudf.DataFrame.from_pandas(edges_df[['src', 'dst', 'length']])
    G = cg.Graph()
    G.from_cudf_edgelist(edges_gdf, source='src', destination='dst', edge_attr='length')
    return G

def find_nearest_nodes_and_calculate_shortest_paths(G, sensors_df, nx_graph):
    distances_df = pd.DataFrame(index=sensors_df['detid'], columns=sensors_df['detid'], data=float('inf'))

    for index, sensor_row in sensors_df.iterrows():
        sensor_lat = sensor_row['lat']
        sensor_long = sensor_row['long']
        sensor_detid = sensor_row['detid']

        # Find nearest node for the sensor
        nearest_node = ox.distance.nearest_nodes(nx_graph, X=sensor_long, Y=sensor_lat)
        sensors_df.at[index, 'node_id'] = nearest_node

        # Compute shortest paths from this node to all others
        df_shortest_paths = cg.sssp(G, nearest_node)

        for _, target_sensor_row in sensors_df.iterrows():
            target_nearest_node = target_sensor_row['node_id']
            target_detid = target_sensor_row['detid']
            if nearest_node != target_nearest_node:
                query_result = df_shortest_paths.query('vertex == @target_nearest_node')
                if not query_result.empty:
                    path_distance = query_result['distance'].values[0]
                    distances_df.at[sensor_detid, target_detid] = path_distance
                    distances_df.at[target_detid, sensor_detid] = path_distance  # Ensure the matrix is symmetric
                else:
                    print(f"No path found from {sensor_detid} to {target_detid}")

    distances_df.replace(float('inf'), 0, inplace=True)
    return distances_df


In [29]:
# Fetch and prepare the road network
place = "Munich, Bavaria, Germany"
nx_graph = fetch_road_network(place, network_type='bike')

# Convert the NetworkX graph to a cuGraph graph
cu_graph = convert_nx_to_cugraph(nx_graph)

<ipython-input-26-ef3d8217d805>:3: FutureWarning: The `utils.config` function is deprecated and will be removed in the v2.0.0 release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True)
/usr/local/lib/python3.10/dist-packages/cugraph/structure/symmetrize.py:93: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(


In [35]:
# Data import path
OS_PATH = os.path.dirname(os.path.realpath('__file__'))
SENSORS_CSV = os.path.join(OS_PATH, '/content/drive/My Drive/data/munich/munich_bicycle_sensors_traffic.csv')

# Import sensors data
sensors_df = pd.read_csv(SENSORS_CSV)
sensors_df = sensors_df.loc[:, ["detid", "lat", "long"]]
sensors_df.dropna(subset=['lat', 'long', 'detid'], how='any', inplace=True)

In [40]:
# Find nearest nodes in the road network and calculate shortest paths using cuGraph
adjacency_matrix = find_nearest_nodes_and_calculate_shortest_paths(cu_graph, sensors_df, nx_graph)

# Output and save the adjacency matrix
print(adjacency_matrix)
adjacency_matrix.to_csv(os.path.join(OS_PATH, '/content/drive/My Drive/output/munich_adjacency_matrix.csv'))

detid         Arnulf  Kreuther   Olympia    Hirsch  Margareten   Erhardt
detid                                                                   
Arnulf         0.000  6313.557  3559.082  3301.919    3551.795  2835.666
Kreuther    6313.557     0.000  8983.361  9603.661    8042.307  3502.520
Olympia     3559.082  8983.361     0.000  4745.086    6366.122  5803.010
Hirsch      3301.919  9603.661  4745.086     0.000    4192.306  6125.770
Margareten  3551.795  8042.307  6366.122  4192.306       0.000  4720.493
Erhardt     2835.666  3502.520  5803.010  6125.770    4720.493     0.000
